In [3]:
!pip install selenium


##Program begining##


## Importing required packages ##

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import time 
import pandas as pd

 ## The Routs for each bus ##

In [2]:
# links for each state

source_links  = [
    "https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/astc/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/uttar-pradesh-state-road-transport-corporation-upsrtc/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/wbtc-ctc/?utm_source=rtchometile"
]


## Scraping route details based on states ##

In [5]:
def route_details(website, state_routes_all): #fuction to pass each state links
    try: # try catch for finding and solving errors
        # arrays to store datas
        names = []
        links = []
        state = []
        # initializing webdriver
        driver = webdriver.Chrome() 
        # passing website to the driver
        driver.get(website)

        # initializing wait element to fetch details efficiently
        wait = WebDriverWait(driver, 10)
        # fetching state name using xpath
        state_name_element = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//a/span[@itemprop="name"]')))

        state_name = state_name_element[2].text
        #fetching route names and route links using xpath
        for i in range(1, 5):
            details = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//a[@class='route']")))
            
            for j in details:
                d = j.get_attribute('href')# fetching link based on href attribute
                e = j.get_attribute('title')# fetching route name based on title attribute
                links.append(d)
                names.append(e)
                state.append(state_name) # storing values into the arrays

            try:
                # click the next page button 
                pagination = wait.until(EC.presence_of_element_located((By.XPATH, '//div[@class="DC_117_paginationTable"]'))) # finding the block of the button
                next_button = pagination.find_element(By.XPATH, f'//div[@class="DC_117_pageTabs " and text()="{i+1}"]') # looping through each button through the block
                driver.execute_script("arguments[0].scrollIntoView();", next_button)# setting up proper view to the button
                wait.until(EC.element_to_be_clickable(next_button)) # waiting for the element to load
                time.sleep(2)
                next_button.click() # clicking the button to pass to the next page
                time.sleep(3) # waiting for the page to load
            except NoSuchElementException: # error handling NoSuchElementException
                print('no such element found', i)

    # handling NoSuchElementException
    except NoSuchElementException:
        print(e)
    
    # creating dictionary 
    result_dict = {
        'routes': names,
        'links': links,
        'state': state
    }
    
    # creating the dataframe using the dict
    df_routes = pd.DataFrame(result_dict)
    state_routes_all = pd.concat([state_routes_all, df_routes], ignore_index=True)
    driver.quit()# quiting the driver

    return state_routes_all

## Calling bus_details to obtain the bus routes names ##


In [9]:
dict_new = {}

state_routes_all  = pd.DataFrame(dict_new, columns=['routes', 'links', 'state']) # final dataframe to store all routes and links

# looping through the states
for i in range(len(source_links)):
    #storing each dataframe result in final dataframe
    state_routes_all = pd.concat([state_routes_all, route_details(source_links[i], state_routes_all)], ignore_index=True)




no such element found 2
no such element found 3
no such element found 4
no such element found 3
no such element found 4
no such element found 4
no such element found 2
no such element found 3
no such element found 4
no such element found 4
no such element found 4


In [11]:
# result dataframe
state_routes_all

,routes,links,state
0,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC
1,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...,APSRTC
2,Hyderabad to Ongole,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC
3,Kakinada to Visakhapatnam,https://www.redbus.in/bus-tickets/kakinada-to-...,APSRTC
4,Bangalore to Tirupati,https://www.redbus.in/bus-tickets/bangalore-to...,APSRTC
...,...,...,...
35433,Habra to Nandakumar (west bengal),https://www.redbus.in/bus-tickets/habra-to-nan...,WBTC (CTC)
35434,Habra to Kolaghat,https://www.redbus.in/bus-tickets/habra-to-kol...,WBTC (CTC)
35435,Kolkata to Mayapur ISKCON,https://www.redbus.in/bus-tickets/kolkata-to-m...,WBTC (CTC)
35436,Habra to Heria,https://www.redbus.in/bus-tickets/habra-to-heria,WBTC (CTC)


## cleaning the dataframe ## 

In [21]:
# droping duplicate datas
state_routes_unique = state_routes_all.drop_duplicates(subset=['routes'], keep='last')

In [23]:
# reseting index
state_routes_unique = state_routes_unique.reset_index()

In [25]:
#droping the unwanted index value
state_routes_unique.drop('index', axis=1, inplace = True)

In [27]:
state_routes_unique

,routes,links,state
0,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...,APSRTC
1,Kakinada to Visakhapatnam,https://www.redbus.in/bus-tickets/kakinada-to-...,APSRTC
2,Bangalore to Tirupati,https://www.redbus.in/bus-tickets/bangalore-to...,APSRTC
3,Bangalore to Kadapa,https://www.redbus.in/bus-tickets/bangalore-to...,APSRTC
4,Ongole to Hyderabad,https://www.redbus.in/bus-tickets/ongole-to-hy...,APSRTC
...,...,...,...
303,Habra to Nandakumar (west bengal),https://www.redbus.in/bus-tickets/habra-to-nan...,WBTC (CTC)
304,Habra to Kolaghat,https://www.redbus.in/bus-tickets/habra-to-kol...,WBTC (CTC)
305,Kolkata to Mayapur ISKCON,https://www.redbus.in/bus-tickets/kolkata-to-m...,WBTC (CTC)
306,Habra to Heria,https://www.redbus.in/bus-tickets/habra-to-heria,WBTC (CTC)


In [30]:
# storing the dataframe to csv
state_routes_unique.to_csv("bus_names_and_routes.csv")

## Bus route details collected ## 